<a href="https://colab.research.google.com/github/divydividivu/DeepLearning.ai-Notes/blob/main/NFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Importing all the header files
from PIL import Image
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import lr_scheduler
from torchvision import transforms, models, datasets
import copy
import os
import cv2
from sklearn.utils import shuffle
from google.colab import drive
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print('GPU is true')
    print('cuda version: {}'.format(torch.version.cuda))
else:
    print('CPU is true')

seed = 3
torch.manual_seed(seed) 

drive.mount('/content/gdrive')

%matplotlib inline
import matplotlib.pyplot as plt

GPU is true
cuda version: 10.2
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
train_bmp2 = os.path.join('/content/gdrive/MyDrive/MSTAR Dataset/MSTAR_PUBLIC_TARGETS_CHIPS_T72_BMP2_BTR70_SLICY/TARGETS/TRAIN/17_DEG/BMP2')
train_btr70 = os.path.join('/content/gdrive/MyDrive/MSTAR Dataset/MSTAR_PUBLIC_TARGETS_CHIPS_T72_BMP2_BTR70_SLICY/TARGETS/TRAIN/17_DEG/BTR70')
train_t72 = os.path.join('/content/gdrive/MyDrive/MSTAR Dataset/MSTAR_PUBLIC_TARGETS_CHIPS_T72_BMP2_BTR70_SLICY/TARGETS/TRAIN/17_DEG/T72')

test_bmp2 = os.path.join('/content/gdrive/MyDrive/MSTAR Dataset/MSTAR_PUBLIC_TARGETS_CHIPS_T72_BMP2_BTR70_SLICY/TARGETS/TEST/15_DEG/BMP2')
test_btr70 = os.path.join('/content/gdrive/MyDrive/MSTAR Dataset/MSTAR_PUBLIC_TARGETS_CHIPS_T72_BMP2_BTR70_SLICY/TARGETS/TEST/15_DEG/BTR70')
test_t72 = os.path.join('/content/gdrive/MyDrive/MSTAR Dataset/MSTAR_PUBLIC_TARGETS_CHIPS_T72_BMP2_BTR70_SLICY/TARGETS/TEST/15_DEG/T72')

In [3]:
label_name = ['BMP2', 'BTR70, T72']
train_img = []
train_label = []

for subclass in os.listdir(train_bmp2):
    for file in os.listdir(train_bmp2 + '/' + subclass):    
        image = cv2.imread(train_bmp2 + '/' + subclass + '/' + file, cv2.IMREAD_GRAYSCALE)
        image = cv2.fastNlMeansDenoising(image, None, 15, 7, 21)
        image = cv2.resize(image, (158, 158))
        train_img.append(image)
        train_label.append([1,0,0])

for subclass in os.listdir(train_btr70):
    for file in os.listdir(train_btr70 + '/' + subclass):    
        image = cv2.imread(train_btr70 + '/' + subclass + '/' + file, cv2.IMREAD_GRAYSCALE)
        image = cv2.fastNlMeansDenoising(image, None, 15, 7, 21)
        image = cv2.resize(image, (158, 158))
        train_img.append(image)
        train_label.append([0,1,0])

for subclass in os.listdir(train_t72):
    for file in os.listdir(train_t72 + '/' + subclass):    
        image = cv2.imread(train_t72 + '/' + subclass + '/' + file, cv2.IMREAD_GRAYSCALE)
        image = cv2.fastNlMeansDenoising(image, None, 15, 7, 21)
        image = cv2.resize(image, (158, 158))
        train_img.append(image)
        train_label.append([0,0,1])

In [4]:
num_epochs = 15
num_classes = 3
batch_size = 32
learning_rate = 3e-4

In [19]:
train_img = np.array(train_img)
train_label = np.array(train_label)

class MyDataset(Dataset):
    def __init__(self, img, label, transform=None):
        super(MyDataset,self).__init__()       
        self.img = torch.from_numpy(img).float()
        self.label = torch.from_numpy(label).long()
        self.transform = transform

    def __getitem__(self, index): 
        img = self.img[index]
        label = self.label[index]
        return img,label

    def __len__(self):
        return self.img.shape[0]

train_dataset = MyDataset(img=train_img,label=train_label, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

In [20]:
for i, (images, labels) in enumerate(train_loader): 
    print(images.shape)
    print(i)
    break


torch.Size([32, 158, 158])
0
